In [95]:
import pandas as pd

In [96]:
df = pd.read_csv('../data/feed-views.log',
                 names=['datetime','user'],
                 sep='\t',
                 engine='python')
df.head()

,datetime,user
0,2020-04-17 12:01:08.463179,artem
1,2020-04-17 12:01:23.743946,artem
2,2020-04-17 12:27:30.646665,artem
3,2020-04-17 12:35:44.884757,artem
4,2020-04-17 12:35:52.735016,artem


In [97]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute
df['second'] = df['datetime'].dt.second
df.head()

,datetime,user,year,month,day,hour,minute,second
0,2020-04-17 12:01:08.463179,artem,2020,4,17,12,1,8
1,2020-04-17 12:01:23.743946,artem,2020,4,17,12,1,23
2,2020-04-17 12:27:30.646665,artem,2020,4,17,12,27,30
3,2020-04-17 12:35:44.884757,artem,2020,4,17,12,35,44
4,2020-04-17 12:35:52.735016,artem,2020,4,17,12,35,52


In [98]:
labels = ['night', 'early morning', 'morning', 'afternoon', 'early evening', 'evening']
bins = [0, 4, 7, 11, 17, 20, 24]
df['daytime'] = pd.cut(df['hour'], labels=labels, bins=bins, include_lowest=True, right=False)
df.set_index('user', inplace=True)
df

,datetime,year,month,day,hour,minute,second,daytime
user,,,,,,,,
artem,2020-04-17 12:01:08.463179,2020,4,17,12,1,8,afternoon
artem,2020-04-17 12:01:23.743946,2020,4,17,12,1,23,afternoon
artem,2020-04-17 12:27:30.646665,2020,4,17,12,27,30,afternoon
artem,2020-04-17 12:35:44.884757,2020,4,17,12,35,44,afternoon
artem,2020-04-17 12:35:52.735016,2020,4,17,12,35,52,afternoon
...,...,...,...,...,...,...,...,...
valentina,2020-05-21 18:45:20.441142,2020,5,21,18,45,20,early evening
maxim,2020-05-21 23:03:06.457819,2020,5,21,23,3,6,evening
pavel,2020-05-21 23:23:49.995349,2020,5,21,23,23,49,evening


In [99]:
df.count()

datetime    1076
year        1076
month       1076
day         1076
hour        1076
minute      1076
second      1076
daytime     1076
dtype: int64

In [100]:
df.value_counts('daytime')

daytime
evening          509
afternoon        252
early evening    145
night            129
morning           36
early morning      5
dtype: int64

In [101]:
df = df.sort_values(['hour','minute','second'])
df

,datetime,year,month,day,hour,minute,second,daytime
user,,,,,,,,
valentina,2020-05-15 00:00:13.222265,2020,5,15,0,0,13,night
valentina,2020-05-15 00:01:05.153738,2020,5,15,0,1,5,night
pavel,2020-05-12 00:01:27.764025,2020,5,12,0,1,27,night
pavel,2020-05-12 00:01:38.444917,2020,5,12,0,1,38,night
pavel,2020-05-12 00:01:55.395042,2020,5,12,0,1,55,night
...,...,...,...,...,...,...,...,...
artem,2020-05-21 23:49:22.386789,2020,5,21,23,49,22,evening
anatoliy,2020-05-09 23:53:55.599821,2020,5,9,23,53,55,evening
pavel,2020-05-09 23:54:54.260791,2020,5,9,23,54,54,evening


In [102]:
max_night = df[df.daytime == 'night']['hour'].max()
print(max_night)
min_morning = df[df.daytime == 'morning']['hour'].min()
print(min_morning)
print(*df[df.hour == max_night].index.unique().to_list())
print(*df[df.hour == min_morning].index.unique().to_list())

3
8
konstantin
alexander


In [103]:
df.hour.mode(), df.daytime.mode()

(0    22
 Name: hour, dtype: int64,
 0    evening
 Name: daytime, dtype: category
 Categories (6, object): ['night' < 'early morning' < 'morning' < 'afternoon' < 'early evening' < 'evening'])

In [104]:
morning_small = df[df.daytime == 'morning'].nsmallest(3, ['hour', 'minute', 'second'])[['hour', 'minute', 'second']]
large = df.nlargest(3, ['hour', 'minute', 'second'])[['hour', 'minute', 'second']]
morning_small, large

(           hour  minute  second
 user                           
 alexander     8      16       3
 alexander     8      35       1
 alexander     9       2      24,
            hour  minute  second
 user                           
 alexander    23      59      38
 valentina    23      58      56
 pavel        23      54      54)

In [107]:
df.describe()

,year,month,day,hour,minute,second
count,1076.0,1076.000000,1076.000000,1076.000000,1076.000000,1076.000000
mean,2020.0,4.870818,13.552974,16.249071,29.629182,29.500929
std,0.0,0.335557,4.906567,6.955490,17.689388,17.405506
min,2020.0,4.000000,1.000000,0.000000,0.000000,0.000000
25%,2020.0,5.000000,11.000000,13.000000,14.000000,14.000000
50%,2020.0,5.000000,13.000000,19.000000,29.000000,30.000000
75%,2020.0,5.000000,15.000000,22.000000,46.000000,45.000000
max,2020.0,5.000000,30.000000,23.000000,59.000000,59.000000


In [108]:
iqr = df.describe()['hour']['75%'] - df.describe()['hour']['25%']
iqr

9.0